In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEvfdey7KRkMVuKEWmaU47URHtRJNbXVz5A4ckhX0amAINA4vz6IZE
Mounted at /content/gdrive


In [3]:
% cd "/content/gdrive/My Drive/efnn"

/content/gdrive/My Drive/efnn


In [4]:
import os
import numpy as np
from tqdm.notebook import tqdm
from efnn.efimg import Exposures

In [5]:
def read_csv(path, delimiter = ','):
    f = open(path, 'r')
    content = f.read().split('\n')
    return [line.split(delimiter) for line in content]

lines = read_csv('./efnn/list2.csv')
print(lines)
# sample name | under ev | normal ev | over ev | use (1) or don't use (0)

[['507', '-8', '-3', '-1', 'ef1', '1'], ['AhwahneeGreatLounge', '-6', '-1', '+1', 'ef3', '1'], ['AirBellowsGap', '-8', '-2', '+0', 'ef4', '1'], ['AmikeusBeaverDamPM1', '-8', '-5', '-2', 'ef5', '1'], ['AmikeusBeaverDamPM2', '-8', '-5', '-2', 'ef5', '1'], ['ArtistPalette', '-8', '-3', '-1', 'ef1', '1'], ['BalancedRock', '-8', '-2', '+0', 'ef4', '1'], ['BandonSunset(1)', '-6', '-2', '+0', 'ef2', '1'], ['BandonSunset(2)', '-6', '-2', '+0', 'ef2', '1'], ['BarHarborPresunrise', '-8', '-2', '+0', 'ef4', '1'], ['BarHarborSunrise', '-8', '-2', '+0', 'ef4', '1'], ['BenJerrys', '-6', '-2', '+0', 'ef2', '1'], ['BigfootPass', '-8', '-3', '-1', 'ef1', '1'], ['BloomingGorse(1)', '-8', '-5', '-2', 'ef5', '0'], ['BloomingGorse(2)', '-8', '-5', '-2', 'ef5', '0'], ['CadesCove', '-8', '-3', '-1', 'ef1', '1'], ['CanadianFalls', '-6', '-1', '+1', 'ef3', '1'], ['CemeteryTree(1)', '-8', '-3', '-1', 'ef1', '1'], ['CemeteryTree(2)', '-8', '-3', '-1', 'ef1', '1'], ['DelicateArch', '-8', '-2', '+0', 'ef4', '1'], 

In [7]:
### OPTION 1 (ALL PIXELS)

SAMPLE_DIR = './dataset/jpg'

for i, line in enumerate(tqdm(lines)):
    if not int(line[5]): 
        # don't use this sample
        continue

    x = Exposures(
        os.path.join(SAMPLE_DIR, line[0]), evs = [
            f'{line[0]}-EV{line[1]}', 
            f'{line[0]}-EV{line[2]}',
            f'{line[0]}-EV{line[3]}', 
        ], colorspace="bgr", ef_file_name = f'{line[4]}.jpg'
    )
    
    x.create_np_data()
    x.save_np_data()

    # take every 3rd pixel
    EVERY_X_PIXEL = 50
    if i == 0:
        data = x.np_data['data'][::EVERY_X_PIXEL]
        labels = x.np_data['labels'][::EVERY_X_PIXEL]
    else:
        data = np.concatenate((data, x.np_data['data'][::EVERY_X_PIXEL]), 0)
        labels = np.concatenate((labels, x.np_data['labels'][::EVERY_X_PIXEL]), 0)

np.save('./new_evs_50th_data.npy', data)
np.save('./new_evs_50th_labels.npy', labels)
print(data[0], labels[0])
print(len(data), len(labels))


[ 0  0  0 16 10  5 37 29 16] [28 19  9]
6560428 6560428


In [ ]:
### OPTION 2 (UNIQUE PIXELS) 
SAMPLE_DIR = './dataset/jpg'

for i, line in enumerate(tqdm(lines)):
    if not int(line[5]): 
        # don't use this sample
        continue

    x = Exposures(
        os.path.join(SAMPLE_DIR, line[0]), evs = [
            f'{line[0]}-EV{line[1]}', 
            f'{line[0]}-EV{line[2]}', 
            f'{line[0]}-EV{line[3]}', 
        ], colorspace="bgr", ef_file_name = f'{line[4]}.jpg'
    )
    
    x.create_np_data()

    y = x.np_data['labels']
    x = x.np_data['data']

    x = np.concatenate((x, y), axis = 1)
    del y

    

    if i == 0:
        data = x[::10]
    else:
        data = np.concatenate((data, x[::10]), 0)

print(data.shape)
data = np.unique(data, axis = 0)
print(data.shape)

np.save('./rgb_data.npy', data[:,:9]) # first 9 pixels are data pixels
np.save('./rgb_labels.npy', data[:,9:]) # last 3 pixels are label pixels


(32802064, 12)
(22228476, 12)


In [ ]:
# x = Exposures(
#     os.path.join(SAMPLE_DIR, lines[0][0]), evs = [
#         f'{lines[0][0]}-EV{lines[0][1]}', 
#         f'{lines[0][0]}-EV{lines[0][2]}', 
#         f'{lines[0][0]}-EV{lines[0][3]}', 
#     ], colorspace="bgr", ef_file_name = f'{lines[0][4]}.jpg'
# )
    

# print("\n", x.pixel(0,1000), data[100], labels[100])
! ls -lh
# ! ls -lh archive/bgr_10th/

total 6.6G
-rw------- 1 root root  25K Jul 15 19:39  15-07-2020-ffnn-1-512-9-5-3-3RD.hdf5
-rw------- 1 root root  25K Jul 15 18:58  15-07-2020-ffnn-1-512-9-5-3.hdf5
-rw------- 1 root root  25K Jul 15 19:54  15-07-2020-ffnn-3-512-9-5-3-3RD.hdf5
-rw------- 1 root root  25K Jul 15 19:12  15-07-2020-ffnn-3-512-9-5-3.hdf5
-rw------- 1 root root  25K Jul 15 20:30  15-07-2020-ffnn-5-512-9-5-3-3RD.hdf5
-rw------- 1 root root  25K Jul 15 19:06  15-07-2020-ffnn-5-512-9-5-3.hdf5
-rw------- 1 root root  32K Jul 15 16:58  15-07-2020.hdf5
-rw------- 1 root root  32K Jul 15 18:33  15-07-2020-linear-10tg.hdf5
-rw------- 1 root root  32K Jul 15 18:38  15-07-2020-linear-10th.hdf5
-rw------- 1 root root  32K Jul 15 17:36  15-07-2020-linear-1-epoch.hdf5
-rw------- 1 root root  32K Jul 15 17:58  15-07-2020-linear.hdf5
-rw------- 1 root root  25K Apr 12 15:24  1hidden.hdf5
-rw------- 1 root root  25K Apr 12 22:31  1hidden_sigmoid.hdf5
-rw------- 1 root root  32K Apr 12 15:04  2hidden.hdf5
-rw------- 1 root 